In [1]:
# Imports 
import numpy as np
import pandas as pd
import os
import pickle

# Image processing imports
from PIL import Image
from tensorflow.keras.utils import load_img
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model

# Progress bar
from tqdm.notebook import tqdm_notebook

# PCA and KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

2023-01-01 21:54:37.484623: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 21:54:47.754691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-01 21:54:47.754952: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-01 21:54:47.754962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

> I'll eventually get around to dealing with these error messages.
>
> Apparently, these are *warnings* not errors.

> The following code is adapted from Gabe Flomo's [tutorial](https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34) on clustering images by similarity.


In [2]:
# Construct the model
model = VGG16()
# Remove the output layer
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
model

2022-12-28 12:16:27.042646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-28 12:16:27.042669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michael-XPS-8700): /proc/driver/nvidia/version does not exist
2022-12-28 12:16:27.042902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 12:16:27.218822: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-12-28 12:16:27.352344: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of fre

In [11]:
# Load and resize the images. The columns of the image_data list is vessel_ID
# followed by image_ID
vessel_ids = []
image_ids = []
images = []
for filename in tqdm_notebook(os.listdir('../images/')):
    try:
        # Open the image, resize it, and convert it to a numpy array
        image = load_img('../images/' + filename)
        image.thumbnail((512, 512))
        image = np.asarray(image)
        
        # Get the vessel id and image id from the filename
        temp = filename.split('_')
        
        # Store the images, features, and ids in lists
        vessel_ids.append(temp[1])
        image_ids.append(temp[2][:-4])
        images.append(image)
    except:
        # Keep track of the files we couldn't open.
        print('Could not open', filename)
        #with open('bad_files.txt', 'a') as f:
        #    f.write(filename + '\n')

  0%|          | 0/9994 [00:00<?, ?it/s]

Could not open drawing_26734_33815.jpg
Could not open drawing_12371_3632.jpg
Could not open drawing_88_99.jpg
Could not open drawing_26462_33470.jpg
Could not open drawing_27751_38822.jpg
Could not open drawing_16094_11216.jpg
Could not open drawing_26431_33501.jpg
Could not open drawing_495_653.jpg
Could not open drawing_15234_9546.jpg
Could not open drawing_15020_9048.jpg
Could not open drawing_15149_9369.jpg
Could not open drawing_26574_33799.jpg
Could not open drawing_26546_33525.jpg
Could not open drawing_26437_33495.jpg
Could not open drawing_11757_2378.jpg
Could not open drawing_27745_38818.jpg
Could not open drawing_26435_33497.jpg
Could not open drawing_27800_38850.jpg
Could not open drawing_27849_38906.jpg
Could not open drawing_26771_33824.jpg
Could not open drawing_27890_38931.jpg
Could not open drawing_12164_3170.jpg
Could not open drawing_26552_33519.jpg
Could not open drawing_16041_11137.jpg
Could not open drawing_26702_33851.jpg
Could not open drawing_15170_9420.jpg
Cou

Could not open drawing_12209_3285.jpg
Could not open drawing_12336_3574.jpg
Could not open drawing_89_100.jpg
Could not open drawing_26536_33536.jpg
Could not open drawing_15965_10880.jpg
Could not open drawing_15110_9286.jpg
Could not open drawing_27872_38918.jpg
Could not open drawing_16079_11196.jpg
Could not open drawing_11756_2375.jpg
Could not open drawing_26759_33785.jpg
Could not open drawing_27892_38932.jpg
Could not open drawing_16053_11157.jpg
Could not open drawing_26667_33730.jpg
Could not open drawing_14716_8142.jpg
Could not open drawing_16240_11447.jpg
Could not open drawing_16077_11194.jpg
Could not open drawing_27858_38909.jpg
Could not open drawing_26643_33754.jpg
Could not open drawing_26646_33751.jpg
Could not open drawing_16054_11159.jpg
Could not open drawing_16108_11241.jpg
Could not open drawing_73_84.jpg
Could not open drawing_26784_33996.jpg
Could not open drawing_486_635.jpg
Could not open drawing_12097_3038.jpg
Could not open drawing_76_87.jpg
Could not ope

Could not open drawing_497_655.jpg
Could not open drawing_15868_10767.jpg
Could not open drawing_26031_33115.jpg
Could not open drawing_26706_33847.jpg
Could not open drawing_485_634.jpg
Could not open drawing_11877_2613.jpg
Could not open drawing_14682_8047.jpg
Could not open drawing_12068_2981.jpg
Could not open drawing_15858_10757.jpg
Could not open drawing_26727_33800.jpg
Could not open drawing_27911_38956.jpg
Could not open drawing_27909_38953.jpg
Could not open drawing_26700_33853.jpg
Could not open drawing_75_86.jpg
Could not open drawing_26708_33845.jpg
Could not open drawing_78_89.jpg
Could not open drawing_358_436.jpg
Could not open drawing_27813_38871.jpg
Could not open drawing_27906_38947.jpg
Could not open drawing_16065_11179.jpg
Could not open drawing_26894_34027.jpg
Could not open drawing_12178_3290.jpg
Could not open drawing_12252_3396.jpg
Could not open drawing_16124_11260.jpg
Could not open drawing_326_394.jpg
Could not open drawing_26800_34058.jpg
Could not open draw

Could not open drawing_12112_3069.jpg
Could not open drawing_26721_33830.jpg
Could not open drawing_16252_11487.jpg
Could not open drawing_15160_9392.jpg
Could not open drawing_367_444.jpg
Could not open drawing_189_215.jpg
Could not open drawing_83_94.jpg
Could not open drawing_27842_38899.jpg
Could not open drawing_26442_33490.jpg
Could not open drawing_11776_2420.jpg
Could not open drawing_364_442.jpg
Could not open drawing_26627_33770.jpg
Could not open drawing_26673_33724.jpg
Could not open drawing_845_1469.jpg
Could not open drawing_12101_3051.jpg
Could not open drawing_26145_33189.jpg
Could not open drawing_15146_9361.jpg
Could not open drawing_67_78.jpg
Could not open drawing_11958_2773.jpg
Could not open drawing_16260_11501.jpg
Could not open drawing_16114_11247.jpg
Could not open drawing_27809_38862.jpg
Could not open drawing_14575_7755.jpg
Could not open drawing_26422_33510.jpg
Could not open drawing_27820_38880.jpg
Could not open drawing_16109_11242.jpg
Could not open drawi

In [41]:
#Reshape the image for use with the VGG16 model
        #image = image.reshape(1, 224, 224, 3)
        # Run the image through the preprocessor
        #image = preprocess_input(image)
        # Construct a feature vector using VGG16
        #feature = model.predict(image)

# Reshape the images
for i in range(len(images)):
    width = images[i].shape[0]
    height = images[i].shape[1]
    images[i] = images[i].reshape(1, width, height, 3)

ValueError: cannot reshape array of size 202752 into shape (1,1,132,3)

In [40]:
# Remove images too small for the vgg16 model
for image in images:
    if image.shape[1] < 32 or image.shape[2] < 32:
        images.remove(image)
        print('Image Removed')

In [43]:
# Preprocess the images
feature_vectors = []
for i in tqdm_notebook(range(len(images))):
    width = images[i].shape[1]
    height = images[i].shape[2]
    model = VGG16(include_top = False, input_shape = (width, height, 3))
    feature_vectors.append(model.predict(images[i]))

  0%|          | 0/9118 [00:00<?, ?it/s]

1/1 [==============================] - 1s 562ms/step


1/1 [==============================] - 0s 377ms/step


1/1 [==============================] - 0s 299ms/step


1/1 [==============================] - 0s 194ms/step


...

1/1 [==============================] - 0s 284ms/step


1/1 [==============================] - 0s 382ms/step


1/1 [==============================] - 0s 234ms/step


1/1 [==============================] - 0s 102ms/step


1/1 [==============================] - 0s 211ms/step


> Instantiating a new model each iteration is a terrible idea. It would be worth taking the time to make all the images a uniform size using a letterbox.

In [54]:
# Store the ids and images in a dataframe
images_df = pd.DataFrame(list(zip(vessel_ids, 
                                  image_ids, 
                                  images, 
                                  feature_vectors)),
                         columns = ['vessel_ID',
                                    'image_ID',
                                    'image',
                                    'feature_vectors'])
images = images_df['image'].copy()

array([[[[ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           1.3557692,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          23.52781  ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          37.306435 ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          33.358307 ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           7.6957793,  0.       ]],

        [[ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.

In [55]:
# Save the data frame
images_df.to_pickle('images.pkl')

In [3]:
# Load the data from
df = pd.read_pickle('images.pkl')
df.head()

,vessel_ID,image_ID,image,feature_vectors
0,18688,15819,"[[[[255 255 255], [255 255 255], [255 255 255]...",[[[[ 0. 0. 0. 0...
1,17496,13961,"[[[[255 255 255], [255 255 255], [255 255 255]...",[[[[ 0. 0. 0. 0...
2,17334,13780,"[[[[255 254 254], [255 254 254], [255 254 254]...",[[[[ 0. 0. 0. 0. ...
3,19762,18235,"[[[[255 255 255], [255 255 255], [255 255 255]...",[[[[ 0. 0. 0. 0...
4,28755,40646,"[[[[255 255 255], [255 255 255], [255 255 255]...",[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ...


In [10]:
# Experimenting
x = df['feature_vectors']
print(x[0].shape, x[1].shape)

# I'm not sure why the dimensions are different. I need to work on this.


(1, 4, 16, 512) (1, 6, 16, 512)


In [ ]:
# Helper function for letterboxing
def letterbox(img, asp_ratio):
    # Heigt > width
    padding = 0.5 * (k * height - asp_ratio / height)
    
    # Height < width
    padding = 0.5 * (k * width - asp_ratio / width)